In [2]:
# imports
from octis.models.CTM import CTM
from octis.dataset.dataset import Dataset
from octis.optimization.optimizer import Optimizer
from skopt.space.space import Real, Categorical, Integer
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.dataset.dataset import Dataset
from skopt.space.space import Real, Categorical, Integer
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.optimization.optimizer import Optimizer
optimizer=Optimizer()
import time
import en_core_web_sm
nlp = en_core_web_sm.load()
dataset = Dataset()
dataset.load_custom_dataset_from_folder("pre_Flower_stop")
docs = [" ".join(words) for words in dataset.get_corpus()]

In [13]:
model = CTM(num_topics=10, bert_path="all-mpnet-base-v2")

search_space = {"num_layers": Integer(1,3), 
                "num_neurons": Categorical({100, 200, 300}),
                "dropout": Real(0.0, 0.95)}

In [14]:
#SETTING METRICS
coherence = Coherence(texts=dataset.get_corpus(), measure = 'c_v')
diversity = TopicDiversity(topk=10)

In [15]:
#RUN CONFIGURATION
optimization_runs=10
model_runs=2

In [16]:
search_space = {"num_layers": Categorical({1, 2, 3}),
                "num_neurons": Categorical({100, 200, 300}),
                "activation": Categorical({'sigmoid', 'relu', 'softplus'}),
                "dropout": Real(0.0, 0.95)
}


In [17]:
#OPTIMIZATION
start = time.time()
optimization_result = optimizer.optimize(
    model, dataset, coherence, search_space, number_of_call=optimization_runs, 
    model_runs=model_runs, save_models=True, 
    extra_metrics=[diversity], # to keep track of other metrics
    save_path='results/test_CTM2//')
end = time.time()
duration = end - start
optimization_result.save_to_csv('results_CTM2.csv')
print('Optimizing model took: ' + str(round(duration)) + ' seconds.')

Current call:  0


KeyboardInterrupt: 

In [ ]:
optimizer = Optimizer()
json_path = 'results/test_CTM2/result.json'
optimizer.resume_optimization(json_path)

In [ ]:
import json
results = json.load(open("results/test_CTM2/result.json",'r'))
results.keys()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.xlabel('Iteration')
plt.ylabel('Coherence score (c_v)')
plt.title('Median coherence score per iteration')
plt.plot(results["f_val"])
plt.show()

In [ ]:
results[ 'f_val'].index(max(results[ 'f_val']))

In [ ]:
print([results['x_iters'][parameter][0] for parameter in results['x_iters'].keys()])